In [81]:
import plotly.express as px
import os
import pandas as pd
from dotenv import load_dotenv

In [1]:
df = pd.read_csv("data/data_file.csv")

In [82]:
load_dotenv()

True

In [83]:
def print_map_column(df, column, differ_size:bool=False, radius:float=0.5, max_size:int=5):
    activity_dict = dict(df[column].value_counts())
    activities = [[v, k] for k, v in activity_dict.items()]
    if differ_size:
        activities.sort()
        activities = [[(index+1)*radius, v[1]] for index, v in enumerate(activities)]
    else:
        activities = [[1*radius, v[1]] for index, v in enumerate(activities)]
    for activity_item in activities:
        activity_dict[activity_item[1]] = activity_item[0]
    df[column+'_rank'] = df[column].map(activity_dict)
    px.set_mapbox_access_token(os.environ.get('MAPBOX_API_TOKEN'))
    fig = px.scatter_mapbox(df, 
                        lat="GPS Y좌표", 
                        lon="GPS X좌표",     
                        color=column, 
                        size=column+"_rank",
                        text='방문지명',
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=max_size, zoom=10)
    fig.show()

In [84]:
print_map_column(df, column='성별', differ_size=True, radius=2, max_size=10)

In [79]:
def print_pie_chart(df:pd.DataFrame, column:str, hole_size:float=0):
    fig = px.pie(df, 
                names=column, 
                title=f'Percentage of {column}')
    fig.update_traces(hoverinfo='label', textinfo='value+percent', textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)), hole=hole_size)
    fig.show()

In [80]:
print_pie_chart(df, '성별', 0.1)

In [37]:
df['동 주소'] = df['지번주소'].apply(lambda x:" ".join(x.split()[:3]))
province_dict = dict(df['동 주소'].value_counts())
province_dict.keys()

province_df = pd.DataFrame({'EMD_KOR_NM':province_dict.keys(), '수치':province_dict.values()})
province_df

,EMD_KOR_NM,수치
0,제주특별자치도 서귀포시 안덕면,1826
1,제주특별자치도 제주시 애월읍,1704
2,제주특별자치도 서귀포시 서귀동,1599
3,제주특별자치도 제주시 구좌읍,1571
4,제주특별자치도 서귀포시 성산읍,1485
...,...,...
77,제주특별자치도 서귀포시 월평동,6
78,제주특별자치도 제주시 화북이동,5
79,제주특별자치도 제주시 추자면,3
80,제주특별자치도 제주시 용강동,2


In [40]:
import json

with open(file='data/jeju.geojson', mode='r', encoding='utf-8') as file:
    be = json.load(file)

In [39]:

fig = px.choropleth(province_df, 
                    geojson=be, 
                    locations="EMD_KOR_NM", 
                    color="수치",
                    featureidkey='properties.EMD_KOR_NM'
                    )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout()
fig.show()